<a href="https://colab.research.google.com/github/kjhyoo/llama2-finetuning/blob/main/chatbot_connect_llama2_finetune_custom_%EB%AA%A8%EB%8D%B8%EB%B6%88%EB%9F%AC%EC%98%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델 불러오기 및 확인 (20240619) * 추가함 (solar모델 기준)

In [ ]:
!pip install accelerate==0.29.3
!pip install bitsandbytes==0.43.1
!pip install transformers==4.40
!pip install peft==0.10.0

In [ ]:
import os

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging
)
from transformers import AutoConfig,AutoModel
import torch
from peft import PeftModel
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# 모델 머지 후 저장 (향후 베이스 모델없이 단독으로 사용가능) T4 GPU 시 OutOfMemory 가능 Colab Pro
base_model = "upstage/SOLAR-10.7B-Instruct-v1.0"

### Load basemodel
baseModel = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,torch_dtype=torch.float16,
    device_map= {"": 0}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.69G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

In [ ]:
### Load basemodel's tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [ ]:
# 저장한 어댑터 모델 불러오기
new_model = "solar-hkcode-model"
basePath = "/content/gdrive/MyDrive/Colab Notebooks/llama"
newModelPath = os.path.join(basePath,new_model)

In [ ]:
# 베이스 모델 및 어댑터 연결
connectedModel = PeftModel.from_pretrained(baseModel, newModelPath)

In [ ]:
# 모델 병합 (메모리 터짐) ColabPro에서만 활용
# mergedModel = connectedModel.merge_and_unload()

In [ ]:
pipe = pipeline(task="text-generation", model=connectedModel, tokenizer=tokenizer, max_length=100)  # 문장 생성을 위한 파이프라인을 설정. 여기서 task는 "text-generation"으로 설정되어 있으므로, 이 파이프라인은 텍스트를 생성하는 데 사용

def extract_response_solar(input, prompt):
    result = pipe(f"### System:\n{input}\n\n### User:\n{prompt}\n\n### Assistant:\n",
                do_sample = True,  # 샘플링 방법을 지정. True : 모델이 확률에 따라 텍스트를 샘플링
                top_p = 0.9,  # 샘플링할 때 사용되는 확률 분포의 상위 일부를 지정
                temperature = 0.1,  # 다양성을 조절
                num_return_sequences = 1,  # 생성할 텍스트 시퀀스의 수를 지정
                eos_token_id = tokenizer.eos_token_id,  # 문장의 끝을 나타내는 토큰의 ID를 지정
                max_length = 4096)  # 생성된 텍스트의 최대 길이

    print(result[0]['generated_text'].split("Assistant:\n")[-1])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyFo

In [ ]:
input = "you are an assistant, answer in korean language"
prompt = "스마트금융과는 어디에 위치하나요?"

extract_response_solar(input, prompt)

서울시 강서구쪽에 위치하며 지하철 5호선 우장산역에서 걸어서 15분 거리에 위치하고 있습니다.


In [ ]:
# 기본 포맷은 solarmodel 사이트 참고 https://huggingface.co/upstage/SOLAR-10.7B-Instruct-v1.0

In [ ]:
import gc

# # Flush memory
# del trainer, model
gc.collect()
torch.cuda.empty_cache()